<a href="https://www.kaggle.com/code/vpotereyko/hw-nn2-fruits-ds?scriptVersionId=298335571" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
import os

# Подивимось на структуру папок датасету
data_path = "/kaggle/input"

for dirname, _, filenames in os.walk(data_path):
    # Показуємо тільки назви папок (класи), не всі файли
    level = dirname.replace(data_path, '').count(os.sep)
    indent = ' ' * 2 * level
    print(f'{indent}{os.path.basename(dirname)}/')
    if level >= 2:  # не заглиблюємось далі 2 рівнів
        continue

input/
  datasets/
    sshikamaru/
      fruit-recognition/
        test/
          test/
        train/
          train/
            Orange/
            Tomato/
            Passion Fruit/
            Cucumber Ripe/
            Cactus fruit/
            Pomegranate/
            Plum/
            Pineapple/
            Papaya/
            Potato Red/
            Kiwi/
            Limes/
            Apple Braeburn/
            Pear/
            Onion White/
            Strawberry/
            Grape Blue/
            Blueberry/
            Apple Granny Smith/
            Apricot/
            Pepper Red/
            Clementine/
            Lemon/
            Avocado/
            Raspberry/
            Cantaloupe/
            Peach/
            Corn/
            Banana/
            Cherry/
            Pepper Green/
            Watermelon/
            Mango/


In [2]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
import matplotlib.pyplot as plt
from torchvision.utils import make_grid

In [3]:
# Трансформації для тренувальних даних (з аугментацією)
# Посилання: https://pytorch.org/vision/stable/transforms.html
# Навіщо це потрібно: нейромережа вчиться краще, коли бачить більше різноманітних прикладів.
# Без аугментації вона може "завчити" конкретні фото замість того,
# щоб зрозуміти загальні ознаки фрукта.
# Аугментація — це коли ти штучно "розмножуєш" свої фотографії, трохи змінюючи кожну.
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),           # Змінюємо розмір до 224x224
    transforms.RandomHorizontalFlip(p=0.5),  # Випадковий горизонтальний переворот
    transforms.RandomRotation(10),           # Випадковий поворот до 10 градусів
    transforms.ToTensor(),                   # Конвертуємо в тензор [0, 1]
])

# Трансформації для тестових даних (без аугментації)
test_transform = transforms.Compose([
    transforms.Resize((224, 224)),           # Тільки змінюємо розмір
    transforms.ToTensor(),                   # Конвертуємо в тензор
])

In [4]:
# Шлях до даних 
# Cтруктура у Блоку 1
train_path = "/kaggle/input/datasets/sshikamaru/fruit-recognition/train/train"
test_path = "/kaggle/input/datasets/sshikamaru/fruit-recognition/test"

# Створюємо датасети з відповідними трансформаціями
train_dataset = datasets.ImageFolder(train_path, transform=train_transform)
test_dataset = datasets.ImageFolder(test_path, transform=test_transform)

print(f"Кількість тренувальних зразків: {len(train_dataset)}")
print(f"Кількість тестових зразків: {len(test_dataset)}")

Кількість тренувальних зразків: 16854
Кількість тестових зразків: 5641
